<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/Standard_banner.png' align='right' width='100%'/>

<font color="#138D75">**NERO Winter School training**</font> <br>
**Copyright:** (c) 2025 EUMETSAT <br>
**License:** GPL-3.0-or-later <br>
**Authors:** Dominika Leskow-Czyżewska (EUMETSAT), based on <a href='https://gitlab.eumetsat.int/eumetlab/data-services/eumdac_data_store/-/blob/master/1_4_Sentinel3_data_access.ipynb'>EUMETSAT Data Services training</a>

<hr>

# Sentinel-3 Data Store access with EUMDAC

### Data used

| Product Description  | Data Store collection ID| Product Navigator |
|:--------------------:|:-----------------------:|:-----------------:|
| Sentinel 3 SLSTR Level 2 Fire Radiative Power | EO:EUM:DAT:0417 | [link](https://data.eumetsat.int/product/EO:EUM:DAT:0417?query=frp&s=extended) |

### Learning outcomes

At the end of this notebook you will know;
* How to refine your <font color="#138D75">**searches**</font> for Sentinel-3 products in the EUMETSAT Data Store using the `eumdac` API client
* How to <font color="#138D75">**download**</font> components of products
* How to pre-screen downloads based on flags

### Outline

Data from Sentinel-3 is available through multiple sources, either via a web user interface (WebUI) or through code and command line interfaces with an Application Programming Interface (API). WebUIs are useful for accessing quick-look data visualisations, and for browsing to see what is available. APIs are more useful for routine, automated and operational data access. Here we will guide you through ways you can access Sentinel-3 data through both methods. We will use OLCI Level-2 Full Resolution data as an example.

1. [Download data via GUIs](#section1)
1. [Authenticating the APIs](#section2)
1. [Example 1: Filter by collections](#section3)
1. [Example 2: Filter by time](#section4)
1. [Example 3: Filter by space and time](#section5)
1. [Example 4: Download by component](#section6)

<hr>

In [1]:
import os
import sys
import pyproj
pyproj.datadir.set_data_dir(os.path.join(sys.prefix, 'share', 'proj'))
import json
import datetime
import shutil
import eumdac
import requests
from IPython.display import YouTubeVideo, HTML

run_name = "varnavas_example"
output_dir = './example_data/'
# Create a download directory for our downloaded products
download_dir = os.path.join(output_dir, run_name, 'Satellite_ActiveFires', 'S3_FRP')
os.makedirs(download_dir, exist_ok=True)

/home/jovyan/nero-winter-school-2025/nero_winter_school/lib/python3.11/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


In [2]:
import credentials

## Downloading data via GUIs

### The EUMETSAT Data Store

The [Data Store](https://data.eumetsat.int) is EUMETSAT's primary pull service for delivering data, including the ocean data available from Sentinel-3. 

Access to it is possible through a WebUI, and through a series of application programming interfaces (APIs). The Data Store supports browsing, searching and downloading data as well as subscription services. It also provides a link to the online version of the [EUMETSAT Data Tailor](https://tailor.eumetsat.int/) for customisation. The video below provides an overview of the Data Store WebUI.

The video below explains how you can access data through the EUMETSAT Data Store WebUI. You can visit and see if you can find Sentinel-3 data to download, however this notebook will also show you how to access the Data Store API, facilitated by the `eumdac` client (see the [Downloading via the Data Store API](#section3) section, below).
#### Links:

* [EUMETSAT Data Store](https://data.eumetsat.int)
* [More information on the Data Store](https://user.eumetsat.int/data-access/data-store)

<div class="alert alert-info" role="alert">

## <a id='section1'></a>Authenticating the APIs
[Back to top](#TOC_TOP)

</div>

To access Sentinel-3 data from the [Data Store](https://data.eumetsat.int), we will use the EUMETSAT Data Access Client (`eumdac`). 

In order to allow us to download data from the Data Store via API, we need to provide our credentials. 

Go to the `.credentials.py` file. In the file, we need to add the following information exactly as follows;

```
EUMDAC_CONSUMER_KEY = "<KEY>"
EUMDAC_CONSUMER_SECRET = "<PASSWORD>"
```

You must replace `<KEY>` and `<PASSWORD>` with the information you extract from https://api.eumetsat.int/api-key/. You will need a [EUMETSAT Earth Observation Portal account](https://eoportal.eumetsat.int/) to access this link, and in order to see the information you must click the "Show hidden fields" button at the bottom of the page.

*Note: your key and secret are permanent, so you only need to do this once, but you should take care to never share them*

Once you have done this, you can read in your credentials using the commands in the following cell. These will be used to generate a time-limited token, which will refresh itself when it expires.

In [3]:
# read your personal key and secret from the credentials file

consumer_key = credentials.EUMDAC_CONSUMER_KEY
consumer_secret = credentials.EUMDAC_CONSUMER_SECRET

credentials = (consumer_key, consumer_secret)
token = eumdac.AccessToken(credentials)

try:
    print(f"This token '{token}' expires {token.expiration}")
except requests.exceptions.HTTPError as exc:
    print(f"Error when tryng the request to the server: '{exc}'")

This token 'b26f1175-c0f3-3bcc-b8e4-36ecce27840a' expires 2025-02-19 23:03:28.886674


After you set your credentials (as described above) you can connect to the service and looking for all Sentinel-3 collection in EUMETSATs DataStore:

In [4]:
# create data store object
datastore = eumdac.DataStore(token)


<div class="alert alert-block alert-success">
<b>NOTE:</b><br />
Find more information about EUMDAC errors, their causes and possible solutions, in our knowledge base: <a href="https://user.eumetsat.int/resources/user-guides/eumetsat-data-access-client-eumdac-guide#ID-Exception-handling">https://user.eumetsat.int/resources/user-guides/eumetsat-data-access-client-eumdac-guide#ID-Exception-handling</a>
</div>

Now we can select our collection of interest, which in this case is EO:EUM:DAT:0417 for SLSTR Level 2 Fire Radiative Power from Sentinel-3.

<div class="alert alert-info" role="alert">

## <a id='section1'></a>Example 1: Filtering by collection
[Back to top](#TOC_TOP)

</div>

Now that we know out collection of interest, we can use `eumdac` to create collection object

In [5]:
# set collection ID for SLSTR L2 FRP
collectionID = 'EO:EUM:DAT:0417'

# Use collection ID
selected_collection = datastore.get_collection(collectionID)
try:
    print(selected_collection.title)
except eumdac.collection.CollectionError as error:
    print(f"Error related to a collection: '{error.msg}'")
except requests.exceptions.RequestException as error:
    print(f"Unexpected error: {error}")

SLSTR Level 2 Fire Radiative Power - Sentinel 3


<div class="alert alert-block alert-success">
<b>NOTE:</b><br />
You can find a Collection ID of a given dataset by searching for it under <a href="https://data.eumetsat.int/">https://data.eumetsat.int/</a>, for example for FRP <a href="https://data.eumetsat.int/product/EO:EUM:DAT:0417">here</a>.
</div>

<div class="alert alert-info" role="alert">

## <a id='section2'></a>Example 2: Filtering by time
[Back to top](#TOC_TOP)

</div>

Now that we specified our collection, we can query the collections it contains. For example, we can search for and list **all** the products that occur within a time range.

In [6]:
# time filter the collection for products
start = "2024-08-11T00:00:00"
end = "2024-08-12T23:59:59"
products = selected_collection.search(dtstart=start, dtend=end)

for product in products:
    try:
        print(product)
    except eumdac.collection.CollectionError as error:
        print(f"Error related to the collection: '{error.msg}'")
    except eumdac.product.ProductError as error:
        print(f"Error related to the product: '{error.msg}'")
    except requests.exceptions.RequestException as error:
        print(f"Unexpected error: {error}")

S3B_SL_2_FRP____20240812T235617_20240813T000117_20240813T011203_0299_096_201______MAR_O_NR_003.SEN3
S3A_SL_2_FRP____20240812T235522_20240813T000022_20240813T015011_0299_115_344______MAR_O_NR_003.SEN3
S3B_SL_2_FRP____20240812T235117_20240812T235617_20240813T011307_0299_096_201______MAR_O_NR_003.SEN3
S3A_SL_2_FRP____20240812T235022_20240812T235522_20240813T015256_0299_115_344______MAR_O_NR_003.SEN3
S3B_SL_2_FRP____20240812T234617_20240812T235117_20240813T011233_0299_096_201______MAR_O_NR_003.SEN3
S3A_SL_2_FRP____20240812T234522_20240812T235022_20240813T015333_0299_115_344______MAR_O_NR_003.SEN3
S3B_SL_2_FRP____20240812T234117_20240812T234617_20240813T011316_0299_096_201______MAR_O_NR_003.SEN3
S3A_SL_2_FRP____20240812T234022_20240812T234522_20240813T015318_0299_115_344______MAR_O_NR_003.SEN3
S3B_SL_2_FRP____20240812T233617_20240812T234117_20240813T011309_0299_096_201______MAR_O_NR_003.SEN3
S3A_SL_2_FRP____20240812T233522_20240812T234022_20240813T015326_0299_115_344______MAR_O_NR_003.SEN3


<div class="alert alert-info" role="alert">

## <a id='section3'></a>Example 3: Filtering by space and time
[Back to top](#TOC_TOP)

</div>

Alongside temporal filtering, we can also pass a bounding box to `eumdac` to specify a region of interest (ROI) that we want to search over.

In [7]:
# space/time filter the collection for products
selected_collection = datastore.get_collection(collectionID)

start_time = "2024-08-11T00:00:00"
end_time = "2024-08-12T23:59:59"

#  lat-lon geographical bounds of search area
W = 23.3
S = 37.8
E = 24.5
N = 38.5

bounding_box = f'{W}, {S}, {E}, {N}'  # West, South, East, North

products = selected_collection.search(
    bbox=bounding_box,
    dtstart=start_time,
    dtend=end_time)

In [8]:
# Print found products
for product in products:
    try:
        print(product)
    except eumdac.collection.CollectionError as error:
        print(f"Error related to the collection: '{error.msg}'")
    except eumdac.product.ProductError as error:
        print(f"Error related to the product: '{error.msg}'")
    except requests.exceptions.RequestException as error:
        print(f"Unexpected error: {error}")

S3A_SL_2_FRP____20240812T200231_20240812T200731_20240812T222811_0299_115_342______MAR_O_NR_003.SEN3
S3B_SL_2_FRP____20240812T080834_20240812T081334_20240812T100843_0299_096_192______MAR_O_NR_003.SEN3
S3A_SL_2_FRP____20240811T202831_20240811T203331_20240811T225320_0299_115_328______MAR_O_NR_003.SEN3
S3B_SL_2_FRP____20240811T195004_20240811T195504_20240811T221523_0299_096_185______MAR_O_NR_003.SEN3
S3A_SL_2_FRP____20240811T090829_20240811T091329_20240811T111510_0299_115_321______MAR_O_NR_003.SEN3
S3B_SL_2_FRP____20240811T082949_20240811T083449_20240811T103531_0299_096_178______MAR_O_NR_003.SEN3


<div class="alert alert-info" role="alert">

## <a id='section4'></a>Example 4: Downloading by component
[Back to top](#TOC_TOP)

</div>

If we do not wish to download an entire product, we can choose to select only a single component using the `entries` property of the product. For example for fires, we are interested only in the `FRP_MWIR1km_standard.csv` file.

In [ ]:
for product in products:
    selected_product = datastore.get_product(product_id=str(product), collection_id=collectionID)
    for entry in selected_product.entries:
        requested_file = 'FRP_MWIR1km_standard.csv'
        if requested_file in entry:
            try:
                target_filename = str(selected_product).split('.')[0] + "_" + requested_file
                with selected_product.open(entry=entry) as fsrc, open(os.path.join(download_dir, target_filename),
                                                                      mode='wb') as fdst:
                    print(f'Downloading {fsrc.name}.')
                    shutil.copyfileobj(fsrc, fdst)
                    print(f'Download of file {fsrc.name} from {selected_product} finished.')
            except eumdac.collection.CollectionError as error:
                print(f"Error related to the collection: '{error.msg}'")
            except eumdac.product.ProductError as error:
                print(f"Error related to the product: '{error.msg}'")
            except requests.exceptions.RequestException as error:
                print(f"Unexpected error: {error}")

Download of file FRP_MWIR1km_standard.csv from S3A_SL_2_FRP____20240812T200231_20240812T200731_20240812T222811_0299_115_342______MAR_O_NR_003.SEN3 finished.
Download of file FRP_MWIR1km_standard.csv from S3B_SL_2_FRP____20240812T080834_20240812T081334_20240812T100843_0299_096_192______MAR_O_NR_003.SEN3 finished.
Download of file FRP_MWIR1km_standard.csv from S3A_SL_2_FRP____20240811T202831_20240811T203331_20240811T225320_0299_115_328______MAR_O_NR_003.SEN3 finished.
Download of file FRP_MWIR1km_standard.csv from S3B_SL_2_FRP____20240811T195004_20240811T195504_20240811T221523_0299_096_185______MAR_O_NR_003.SEN3 finished.
Download of file FRP_MWIR1km_standard.csv from S3A_SL_2_FRP____20240811T090829_20240811T091329_20240811T111510_0299_115_321______MAR_O_NR_003.SEN3 finished.


<div class="alert alert-info" role="alert">

## <a id='section5'></a>Example 5: Convert to shapefile
[Back to top](#TOC_TOP)

</div>

It is possible to load each csv file in the a GIS software. However, it is much more convenient to combine all the files into one shapefile.

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely import wkt
from shapely.geometry import box

In [ ]:
#List all the files in a given path. Those files will be combined in one shapefile
folder_path = download_dir
csv_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if 'csv' in f]
csv_files

First, we concatenate all the csv files into one.

In [ ]:
# Set the number of header lines to skip
header_lines = 19

# Initialize an empty list to store GeoDataFrames
gdfs = []

for csv_file in csv_files:

    # Read the CSV file
    df = pd.read_csv(csv_file, skiprows=header_lines)

    # Convert the DataFrame to a GeoDataFrame
    gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df['lon(deg)'], df['lat(deg)']))

    # Set the CRS to WGS84 (EPSG:4326)
    gdf.set_crs(epsg=4326, inplace=True)

    # Filter points within the bounding_box 
    bounding_box_tuple = tuple(map(float, bounding_box.split(',')))
    bbox = box(*bounding_box_tuple)
    gdf = gdf[gdf.within(bbox)]

    # Append the GeoDataFrame to the list
    gdfs.append(gdf)

# Concatenate all GeoDataFrames into a single GeoDataFrame
big_gdf = pd.concat(gdfs, ignore_index=True)

print(big_gdf)


To distinguish easily which points were burning when, we combine day and time columns into one.

In [ ]:
# Combine day and time columns, skip seconds
big_gdf['day_time'] = big_gdf['day'] + ' ' + big_gdf['time'].str.slice(0, 5)

Finally, we can save the resulting shapefile.

In [ ]:
# Save all columns as properties in the shapefile
shapefile_path = os.path.join(folder_path, os.path.basename(requested_file).split('.')[0] + '.shp')
big_gdf.to_file(shapefile_path, driver='ESRI Shapefile')

print(f"Shapefile saved to {shapefile_path}")

<div class="alert alert-block alert-success">
<b>NOTE:</b><br />
Shapefiles can have columns only up to 10 characters long. Therefore, you probably received a warning about the column names trucation. The warning does not impact quality of the resulting file.
</div>

<hr>

<p style="text-align:left;">This project is licensed under the <a href="./LICENSE.TXT">GPL-3.0-or-later</a> license <span style="float:right;"><a href="https://gitlab.eumetsat.int/eumetlab/atmosphere/trainings/nero-winter-school-2025">View on GitLab</a> | <a href="https://classroom.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact</a></span></p>